# CbO

In [52]:
G = {1, 2, 3, 4, 5}
M = {'a', 'b', 'c', 'd'}
M_G = {
    'a': {2, 4, 5},
    'b': {1, 2, 3, 5},
    'c': {1, 2, 3},
    'd': {1, 4, 5},
}
G_M = {
    1: {'b', 'c', 'd'},
    2: {'a', 'b', 'c'},
    3: {'b', 'c'},
    4: {'a', 'd'},
    5: {'a', 'b', 'd'}
}

In [102]:
def print_L(L):
    print('\tL = {')
    for l in L:
        print('\t\t(' + str(set(l[0])), str(set(l[1])) + ')', sep=',', end=',\n')
    print('\t}')

In [103]:
def derivative(g, formal_type='extent'):

    if len(g) == 0:
        if formal_type == 'extent':
            return M
        if formal_type == 'intent':
            return G
    
    if next(iter(g)) in M:
        find_dict = M_G.copy()
        der = G.copy()
    else:
        find_dict = G_M.copy()
        der = M.copy()
    
    for i in g:
        der &= find_dict[i]

    return der
        

In [124]:
def process(A, g, C, D, L):
    print('Process(' + str(A), g, C, str(D) + ')', sep=', ')
    print('\t', {h for h in C.difference(A) if g < h}, ' = { }', sep='')
    if len({h for h in C.difference(A) if g < h}) == 0:
        L |= {(frozenset(C), frozenset(D))}
        print_L(L)
    
    print('\tfor f in',  {h for h in G.difference(A) if g < h})
    for f in {h for h in G.difference(A) if g < h}:
        print('\tf =', f)
        Z = C | {f}
        print('\tZ =', Z)
        Y = D & derivative({f}, formal_type='extent')
        print('\tf_der = ', derivative({f}, formal_type='extent'))
        print('\tY =', Y)
        X = derivative(Y, formal_type='intent')
        print('\tX =', X)
        process(Z, f, X, Y, L)

L = set()
for g in G:
    g_ = g
    g_der = derivative({g_})
    process({g_}, g_, derivative(g_der), g_der, L)    

Process({1}, 1, {1}, {'c', 'b', 'd'})
	set() = { }
	L = {
		({1},{'c', 'b', 'd'}),
	}
	for f in {2, 3, 4, 5}
	f = 2
	Z = {1, 2}
	f_der =  {'c', 'b', 'a'}
	Y = {'c', 'b'}
	X = {1, 2, 3}
Process({1, 2}, 2, {1, 2, 3}, {'c', 'b'})
	{3} = { }
	for f in {3, 4, 5}
	f = 3
	Z = {1, 2, 3}
	f_der =  {'c', 'b'}
	Y = {'c', 'b'}
	X = {1, 2, 3}
Process({1, 2, 3}, 3, {1, 2, 3}, {'c', 'b'})
	set() = { }
	L = {
		({1, 2, 3},{'c', 'b'}),
		({1},{'c', 'b', 'd'}),
	}
	for f in {4, 5}
	f = 4
	Z = {1, 2, 3, 4}
	f_der =  {'a', 'd'}
	Y = set()
	X = {1, 2, 3, 4, 5}
Process({1, 2, 3, 4}, 4, {1, 2, 3, 4, 5}, set())
	{5} = { }
	for f in {5}
	f = 5
	Z = {1, 2, 3, 4, 5}
	f_der =  {'b', 'd', 'a'}
	Y = set()
	X = {1, 2, 3, 4, 5}
Process({1, 2, 3, 4, 5}, 5, {1, 2, 3, 4, 5}, set())
	set() = { }
	L = {
		({1, 2, 3},{'c', 'b'}),
		({1},{'c', 'b', 'd'}),
		({1, 2, 3, 4, 5},set()),
	}
	for f in set()
	f = 5
	Z = {1, 2, 3, 5}
	f_der =  {'b', 'd', 'a'}
	Y = {'b'}
	X = {1, 2, 3, 5}
Process({1, 2, 3, 5}, 5, {1, 2, 3, 5}, {'b'})

# Equvalence classes

In [110]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [119]:
def print_eq(eq):
    for e in eq:
        print(set(e), eq[e])

In [120]:
power_M = list(set(x) for x in powerset(M))
eq_classes = {}
for s in power_M:
    s_der = frozenset(derivative(s, formal_type='intent'))
    if s_der in eq_classes:
        eq_classes[s_der].append(s)
    else:
        eq_classes[s_der] = [s]

print_eq(eq_classes)


{1, 2, 3, 4, 5} [set()]
{2, 4, 5} [{'a'}]
{1, 2, 3} [{'c'}, {'c', 'b'}]
{1, 2, 3, 5} [{'b'}]
{1, 4, 5} [{'d'}]
{2} [{'c', 'a'}, {'c', 'b', 'a'}]
{2, 5} [{'b', 'a'}]
{4, 5} [{'d', 'a'}]
{1} [{'c', 'd'}, {'c', 'b', 'd'}]
{1, 5} [{'b', 'd'}]
set() [{'c', 'd', 'a'}, {'c', 'b', 'd', 'a'}]
{5} [{'b', 'd', 'a'}]


# Generator implication cover

In [122]:
nmingen = [{'c'}, {'c', 'a'}, {'c', 'd'}, {'c', 'd', 'a'}]
for f in nmingen:
    print(f, '\\rightarrow', derivative(derivative(f)).difference(f))

{'c'} \rightarrow {'b'}
{'c', 'a'} \rightarrow {'b'}
{'c', 'd'} \rightarrow {'b'}
{'c', 'd', 'a'} \rightarrow {'b'}
